In [ ]:
import json
import pandas as pd
def read_json_nrows(nrows,filename):
    n= 0
    with open(filename) as f:
        while n < nrows:
            if n == 0:
                line = f.readline()
                line = json.loads(line.rstrip())
                train = pd.DataFrame(line,index = [0])
            else:
                temp = pd.DataFrame(json.loads(f.readline().rstrip()),index = [n])
                train = train.append(temp)
            n = n+1
    return train
    

In [ ]:
import re
import numpy as np 
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.stem import SnowballStemmer,WordNetLemmatizer
stemmer=SnowballStemmer('english')
lemma=WordNetLemmatizer()
from string import punctuation
import nltk
import os
import gc
from keras.preprocessing import sequence,text
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense,Dropout,Embedding,LSTM,Conv1D,GlobalMaxPooling1D,Flatten,MaxPooling1D,GRU,SpatialDropout1D,Bidirectional
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score

In [ ]:
train = pd.read_json('review_train.json',orient = 'records',lines = True)
test = pd.read_json('review_test.json',orient = 'records',lines = True)

In [ ]:
num_samples = 1000000

In [ ]:
train = train[:num_samples]
train_eng = train

In [ ]:
def preprocessing(text):
    # 取表情
    # emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)
    # 去回车
    text = re.sub('\\n',' ',text)
    # not
    # text = no_abbreviation(text)
    # 只保留字母
    text = re.sub('[\W]+',' ', text.lower())
    # 统一转折词
    # text = change_adversatives(text)
    # 词性还原
    # tokens = lemmatizer(text)
    # text = ''
    # for index, token in enumerate(tokens):
        # if token in stop:
        #     tokens[index] = ''
        # else:
        #     text = text + tokens[index] + ' '
    # return {'text':text,'emoticons':emoticons}
    return {'text':text}

In [ ]:
from tqdm import tqdm, tqdm_pandas
tqdm.pandas()
dictionary_train = train_eng.text.progress_apply(preprocessing)
dictionary_test = test.text.progress_apply(preprocessing)

In [ ]:
texts_train = [dictionary_train[i]['text'] for i in train_eng.index]
texts_test = [dictionary_test[i]['text'] for i in test.index]

In [ ]:
import gc
del train_eng
del test
del train
del dictionary_train
del dictionary_test
gc.collect()

In [ ]:
num_train = len(texts_train)
num_test = len(texts_test)

In [ ]:
texts_train.extend(texts_test)

In [ ]:
del texts_test
gc.collect()

In [ ]:
texts = texts_train

In [ ]:
del texts_train
gc.collect()

In [ ]:
new_texts = texts

In [ ]:
del texts
gc.collect()

In [ ]:
from gensim.models.phrases import Phrases, Phraser

In [ ]:
sentence_stream = [sent.split(' ') for sent in tqdm(new_texts)]

In [ ]:
bigram = Phraser(Phrases(sentence_stream, min_count=5, threshold=5)) #mincount越小识别出来的越少，threshold higher means fewer phrases

In [ ]:
sentence_with_phrase = bigram[sentence_stream]

In [ ]:
result = ['']
for i in tqdm(range(len(new_texts))):
    result.append(' '.join(bigram[sentence_stream[i]]))
    
new_texts = result[1:]

In [ ]:
del sentence_stream
del result
gc.collect()

In [ ]:
len(sentence_with_phrase)

In [ ]:
new_texts

#### W2V

In [ ]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [ ]:
from gensim.models import Word2Vec
w2v_model = Word2Vec(min_count=1,
                     window=2,
                     size=200,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores,
                     seed = 123,
                     sg=0) # sg默认为0，对应CBOW算法；sg=1则采用skip-gram算法

In [ ]:
import time

t = time.clock()

w2v_model.build_vocab(sentence_with_phrase, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time.clock() - t) / 60, 2)))

In [ ]:
t = time.clock()

w2v_model.train(sentence_with_phrase, total_examples=w2v_model.corpus_count, epochs=20, report_delay=1)

print('Time to train the model: {} mins'.format(round((time.clock() - t) / 60, 2)))

In [ ]:
pd.DataFrame(columns=["t"],data = new_texts).to_csv('./first_1000000.csv')

In [ ]:
new_texts = pd.read_csv('./first_1000000.csv')

In [ ]:
new_texts = new_texts["t"].values
new_texts[np.argwhere(pd.isnull(new_texts))] = ''

In [ ]:
original_y = pd.read_csv("thirtyM_y.csv",header=None).values
y = original_y.flatten()
y = y[:num_samples]

#### deep learning

In [ ]:
train_text = new_texts[:num_samples]
test_text = new_texts[num_samples:]
y = pd.get_dummies(y).values # one_hot
print(train_text.shape,test_text.shape,y.shape)

In [ ]:
X_train_text,X_val_text,y_train,y_val=train_test_split(train_text,y,test_size=0.2,stratify=y,random_state=123)
print(len(X_train_text),y_train.shape)
print(len(X_val_text),y_val.shape)

In [ ]:
all_words=' '.join(new_texts)
all_words=word_tokenize(all_words)
dist=FreqDist(all_words)
num_unique_word=len(dist)
num_unique_word

In [ ]:
r_len=[]
for text in new_texts:
    word=word_tokenize(text)
    l=len(word)
    r_len.append(l)
    
max_review_len=np.max(r_len)
max_review_len

#### LSTM

In [ ]:
max_features = num_unique_word
max_words = max_review_len
batch_size = 1280
epochs = 1
num_classes=5

In [ ]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train_text)


In [ ]:
X_train = tokenizer.texts_to_sequences(X_train_text) #得到句中每个词的索引,由索引组成data
X_val = tokenizer.texts_to_sequences(X_val_text)


In [ ]:
X_test = tokenizer.texts_to_sequences(test_text)

In [ ]:
X_train = sequence.pad_sequences(X_train, maxlen=max_words) #每行有max_words列，不全的在前段补成0
X_val = sequence.pad_sequences(X_val, maxlen=max_words)


In [ ]:
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

In [ ]:
print(X_train.shape,X_val.shape,X_test.shape)

In [ ]:
model_lstm=Sequential()
model_lstm.add(Embedding(max_features,100,mask_zero=True)) #emdding层负责W2V
#input_dim: smaples = 100000, timesteps = max_words即一句话的词个数，统一了已经, output_dim = 100即词向量的列数
model_lstm.add(LSTM(64,dropout=0.4, recurrent_dropout=0.4,return_sequences=True)) 
model_lstm.add(LSTM(32,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model_lstm.add(Dense(num_classes,activation='softmax'))
model_lstm.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model_lstm.summary()

In [ ]:
from keras.models import load_model
model_lstm = load_model('model_lstm.h5')

In [ ]:
f_lstm=model_lstm.fit(X_train, y_train, validation_data=(X_val, y_val),
                    epochs=epochs, batch_size=batch_size, verbose=1)

In [ ]:
model_lstm.save('model_lstm.h5')

In [ ]:
y_pred_lstm=model_lstm.predict_classes(X_test,verbose=1)

In [ ]:
id = np.array(range(1,len(y_pred)+1))
header = np.array([['Id','Expected']])
y_pred = y_pred_lstm.reshape([-1,1])
id = id.reshape([-1,1])
ans = np.hstack((id, y_pred))
ans = np.vstack((header, ans))
np.savetxt("TueG1_submit3.csv", ans, delimiter=",", fmt='%s')

loss,accuracy = model.evaluate(x_test,y_test)

print('\ntest loss',loss)
print('accuracy',accuracy)

#### LSTM+CNN

In [ ]:
print(X_train.shape,X_val.shape,X_test.shape)

In [ ]:
max_features = num_unique_word
max_words = max_review_len
batch_size = 1280
epochs = 1
num_classes=5

In [ ]:
max_features = 498426
max_words = 1058

In [ ]:
model_lstm_cnn = Sequential()
model_lstm_cnn.add(Embedding(max_features,100,input_length=max_words))
model_lstm_cnn.add(Dropout(0.2))
model_lstm_cnn.add(LSTM(32,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
model_lstm_cnn.add(Conv1D(64,kernel_size=3,padding='valid',activation='relu'))
model_lstm_cnn.add(MaxPooling1D(pool_size=2))
model_lstm_cnn.add(Dropout(0.25))
model_lstm_cnn.add(Flatten())
model_lstm_cnn.add(Dense(64,activation='relu'))
model_lstm_cnn.add(Dropout(0.5))
model_lstm_cnn.add(Dense(5,activation='softmax'))

In [ ]:
model_lstm_cnn.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])

In [ ]:
print(model_lstm_cnn.summary())

embedding: 49842600 = 498426*100，输出1058的句子长度，100维的embedding size, 即1058*100

lstm: 17042 = 4*32*((100+32)+1)即4个gate, 32个unit, 上一次的状态 h(t-1)是怎么和下一次的输入 x(t) 结合（concat）起来的, 就是把二者直接拼起来，x是100维的向量，h(t-1)是32维的，那么拼起来就是132位的向量，加上1个bias, 输出1058的句子长度，32个LSTM的unit，每个有一个最后输出,所以是1058*32

1Dconv: 6208 = 64*(3*32 + 1), 64个1维卷积核(图片用2维), 每个size是3，每行都有32个参数，1是bias。输出是kernel每走一步与区域乘出来的值relu后都加起来等到一个和，一共走1056次，头尾没法走，所以输出是1056*64，可以做zero-padding去变成1058.

maxpooling: 用kernal size一半大小的区域来找每个区域的最大值，从而筛选掉一半变成528*64

flatten: 就是变成了528*64 = 33792个输出

fully connected层：2162752 = 64*(33792+1),即64个神经元，每个有33792个w和1个bias，relu后输出每个神经元一个值，也就是64个

softmax层：5 class label，共有64*(5+1) = 325个参数，所以是输出5个概率值

In [ ]:
from keras.models import load_model
model_lstm_cnn = load_model('model_lstm_cnn.h5')

In [ ]:
model_lstm_cnn.fit(X_train, y_train, validation_data=(X_val, y_val),
                    epochs=2, batch_size=batch_size, verbose=1)

In [ ]:
model_lstm_cnn.save('model_lstm_cnn.h5')

In [ ]:
from keras.models import load_model
model_lstm_cnn = load_model('model_lstm_cnn.h5')

In [ ]:
model_lstm_cnn.fit(X_train, y_train, validation_data=(X_val, y_val),
                    epochs=epochs, batch_size=batch_size, verbose=1)

In [ ]:
model_lstm_cnn.save('model_lstm_cnn_3.h5')

In [ ]:
y_pred=model_lstm_cnn.predict_classes(X_test, verbose=1)

In [ ]:
y_pred = y_pred+1

In [ ]:
id = np.array(range(1,len(y_pred)+1))
header = np.array([['Id','Expected']])
y_pred = y_pred.reshape([-1,1])
id = id.reshape([-1,1])
ans = np.hstack((id, y_pred))
ans = np.vstack((header, ans))
np.savetxt("TueG1_submit7.csv", ans, delimiter=",", fmt='%s')

In [ ]:
model_lstm_cnn = load_model('model_lstm_cnn_3.h5')

0.72 using LSTM+CNN+categorical_cross_entrophy+3epoch

#### Bi-LSTM+Attention